In [22]:
#import the library
import scanpy as sc
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sb
import seaborn as sns
import logging
import os
import importlib
import warnings
warnings.filterwarnings("ignore")
os.chdir(r'E:\AAA_Labwork\Tcell tissues\v2')
import pickle as pkl
from matplotlib.colors import LinearSegmentedColormap

import re
from gprofiler import GProfiler
import rpy2.rinterface_lib.callbacks
from rpy2.robjects import pandas2ri
import anndata2ri

In [2]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [3]:
#This pallete is for colorblinds
my_palette = ['#0351A8','#8CB0E0','#D56D11','#FFBB78','#234E08','#53CB8B','#D30083','#CB788D','#4E195A','#C58CCF','#AA290F','#B03FD1','#E8BCCF','#64605F','#B2AD9A','#D2D30B','#D1BD4F','#06DCF2','#9EDAE5','#517219','#5B43CF','#D92F24','#FFD900','#002F33','#B8A3A3']

In [4]:
from matplotlib.colors import LinearSegmentedColormap
values = [0, 1]
colors = [(227, 227, 227), (255, 42, 18)]
colors = [tuple(np.array(color) / 255) for color in colors]
my_cmap = LinearSegmentedColormap.from_list('', colors)

In [25]:
# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)

In [26]:
%%R
# Load libraries from correct lib Paths for my environment - ignore this!
.libPaths(.libPaths('C:\\Users\\16220\\AppData\\Local\\R\\win-library\\4.3'))

# Load all the R libraries we will be using in the notebook
library(scran)
library(Matrix)
library(Seurat)
library(RColorBrewer)
library(slingshot)
library(monocle)
library(gam)
library(ggplot2)
library(plyr)
library(MAST)
library(clusterExperiment)
library(SingleCellExperiment)
library(monocle3)
library(destiny)
library(circlize)

Exception ignored from cffi callback <function _consolewrite_ex at 0x00000231A13DA670>:
Traceback (most recent call last):
  File "c:\Users\16220\anaconda3\envs\cpdb\lib\site-packages\rpy2\rinterface_lib\callbacks.py", line 133, in _consolewrite_ex
    s = conversion._cchar_to_str_with_maxlen(buf, n, _CCHAR_ENCODING)
  File "c:\Users\16220\anaconda3\envs\cpdb\lib\site-packages\rpy2\rinterface_lib\conversion.py", line 138, in _cchar_to_str_with_maxlen
    s = ffi.string(c, maxlen).decode(encoding)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd4 in position 0: invalid continuation byte
Exception ignored from cffi callback <function _consolewrite_ex at 0x00000231A13DA670>:
Traceback (most recent call last):
  File "c:\Users\16220\anaconda3\envs\cpdb\lib\site-packages\rpy2\rinterface_lib\callbacks.py", line 133, in _consolewrite_ex
    s = conversion._cchar_to_str_with_maxlen(buf, n, _CCHAR_ENCODING)
  File "c:\Users\16220\anaconda3\envs\cpdb\lib\site-packages\rpy2\rinterface_lib\


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

Exception ignored from cffi callback <function _consolewrite_ex at 0x00000231A13DA670>:
Traceback (most recent call last):
  File "c:\Users\16220\anaconda3\envs\cpdb\lib\site-packages\rpy2\rinterface_lib\callbacks.py", line 133, in _consolewrite_ex
    s = conversion._cchar_to_str_with_maxlen(buf, n, _CCHAR_ENCODING)
  File "c:\Users\16220\anaconda3\envs\cpdb\lib\site-packages\rpy2\rinterface_lib\conversion.py", line 138, in _cchar_to_str_with_maxlen
    s = ffi.string(c, maxlen).decode(encoding)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd4 in position 1: invalid continuation byte
Exception ignored from cffi callback <function _consolewrite_ex at 0x00000231A13DA670>:
Traceback (most recent call last):
  File "c:\Users\16220\anaconda3\envs\cpdb\lib\site-packages\rpy2\rinterface_lib\callbacks.py", line 133, in _consolewrite_ex
    s = conversion._cchar_to_str_with_maxlen(buf, n, _CCHAR_ENCODING)
  File "c:\Users\16220\anaconda3\envs\cpdb\lib\site-packages\rpy2\rinterface_lib\

In [5]:
adata_gut = sc.read_h5ad(r"gut_annotated.h5ad")
adata_liver = sc.read_h5ad(r"liver_annotated.h5ad")

### answer the question of what cell types contributes to the donor T/mac difference

In [17]:
adata_DEG_gut_T = adata_gut[(adata_gut.obs['celltype_lowres'].isin(['T Cells']))&(adata_gut.obs['batch'].isin(['3','4']))]
adata_DEG_gut_Mac = adata_gut[(adata_gut.obs['celltype_lowres'].isin(['Macrophages']))&(adata_gut.obs['batch'].isin(['3','4']))]

In [15]:
adata_DEG_liver_T = adata_liver[(adata_liver.obs['celltype_lowres'].isin(['T Cells']))&(adata_liver.obs['batch'].isin(['3','4']))]
adata_DEG_liver_Mac = adata_liver[(adata_liver.obs['celltype_lowres'].isin(['Macrophages']))&(adata_liver.obs['batch'].isin(['3','4']))]

In [21]:
adata_list = [adata_DEG_gut_T,adata_DEG_gut_Mac,adata_DEG_liver_T,adata_DEG_liver_Mac]

In [28]:
names = ['gutT','gutMac','liverT','liverMac']
for ind, i in enumerate(adata_list):
    colnames = i.obs_names
    rownames = i.var_names
    counts = i.layers['counts'].T

    donorID = i.obs['Donor ID']
    name = names[ind]

    %R setwd('E:/AAA_Labwork/Tcell tissues/v2')
    %R -i counts -i colnames -i rownames -i donorID -i name
    %R colnames(counts) = colnames
    %R rownames(counts) = rownames
    %R srat <- CreateSeuratObject(counts = counts,project = name, min.cells = 0, min.features = 0)
    %R srat[['DonorID']] = donorID
    %R saveRDS(srat, file = name)
